In [2]:
import json
import pandas as pd
import requests
import time

import api_utils

import matplotlib.pyplot as plt
plt.style.use('seaborn')

C:\Users\voyno\AppData\Local\Temp\ipykernel_6912\1969594201.py:9: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [3]:

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.chrome.options import Options


options = Options()
options.add_argument('--headless')


def create_driver(link=None):
    """
    Create chrome driver, get link if available
    """
    driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)
    if link:
        driver.get(link)

    return driver

In [73]:
data = []

resp = api_utils.get_76ers_games()
urls = [event['url'] for event in resp['events']]

driver = create_driver()
for url in urls:
    print(url)

    driver.get(url)
    driver.maximize_window()
    driver.execute_script("document.body.style.zoom='20%'")
    time.sleep(1)

    text = driver.find_element(By.ID, '__next').text
    data.append(text.split("\n"))

driver.quit()


C:\Users\voyno\AppData\Local\Temp\ipykernel_6912\2241503389.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)
C:\Users\voyno\AppData\Local\Temp\ipykernel_6912\2241503389.py:17: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)


https://seatgeek.com/nets-at-76ers-tickets/1-25-2023-philadelphia-pennsylvania-wells-fargo-center/nba/5775973
https://seatgeek.com/nuggets-at-76ers-tickets/1-28-2023-philadelphia-pennsylvania-wells-fargo-center/nba/5775975
https://seatgeek.com/magic-at-76ers-tickets/1-30-2023-philadelphia-pennsylvania-wells-fargo-center/nba/5775977
https://seatgeek.com/magic-at-76ers-tickets/2-1-2023-philadelphia-pennsylvania-wells-fargo-center/nba/5775979
https://seatgeek.com/knicks-at-76ers-tickets/2-10-2023-philadelphia-pennsylvania-wells-fargo-center/nba/5775981
https://seatgeek.com/rockets-at-76ers-tickets/2-13-2023-philadelphia-pennsylvania-wells-fargo-center/nba/5775983
https://seatgeek.com/cavaliers-at-76ers-tickets/2-15-2023-philadelphia-pennsylvania-wells-fargo-center/nba/5775985
https://seatgeek.com/grizzlies-at-76ers-tickets/2-23-2023-philadelphia-pennsylvania-wells-fargo-center/nba/5775987
https://seatgeek.com/celtics-at-76ers-tickets/2-25-2023-philadelphia-pennsylvania-wells-fargo-center/

In [78]:
clean_data = {}
title_string = " at Philadelphia 76ers"
for dataset in data:

    if "Aisle Seats" in dataset:
        dataset.remove("Aisle Seats")

    if "Private Restrooms" in dataset:
        dataset.remove("Private Restrooms")
    
    if "Includes access to a buffet, Includes Parking" in dataset:
        dataset.remove("Includes access to a buffet, Includes Parking")
        
    for row in dataset:
         if title_string in row:
            title = row.replace(title_string, "")
            break

    while title in clean_data:
        title += "(1)"
    clean_data[title] = dataset
    

tokens = ["Sorted", "Returnable"]
for k, v in clean_data.items():

    idxs = []
    for i, item in enumerate(v):
        for token in tokens:
            if token in item:
                idxs.append(i)
    maxidx = max(idxs) + 1

    if "$" in "".join(v[:maxidx]):
        raise ValueError("(Validation Warning) Attempting to remove valid data")
    clean_data[k] = v[maxidx:]

In [81]:
cols = ["price", "score", "deal", "num_tickets", "section"]

dfs = {}
for k, v in clean_data.items():
    data_collection = {col: [] for col in cols}
    for i, item in enumerate(v):
        col = cols[i % 5]
        data_collection[col].append(item)
    
    try:
        dfs[k] = pd.DataFrame(data_collection)
    except ValueError:
        print(f"Failed on {k}")


Failed on Boston Celtics
Failed on Miami Heat


In [84]:
for i, item in enumerate(clean_data["Boston Celtics"]):
    if not i % 5:
        print(item)

$2,881 each
Includes access to a buffet, Includes Parking
Section 123, Row 5
Section 214, Row 2
Section 113, Row 17
Section 102, Row BB
Section 211, Row 2
Section 112, Row 13
Section 111, Row 13
Section 113, Row 2
Section 119, Row 8
Section 110, Row 15
Section R 106, Row J
Section 124, Row 14
